In [ ]:
import matplotlib as mpl
mpl.use('Agg')

from matplotlib import rcParams
rcParams['font.family']='serif'
rcParams['mathtext.fontset'] = 'dejavuserif'

import matplotlib.pyplot as plt

import massmapper as mm
import numpy as np
import random
import h5py

%matplotlib inline

In [ ]:
masswidth=0.5
catalog_kind='subhalo'
dat12=mm.Mergerdata(12.,0.5,catalog_kind=catalog_kind)
dat12.tracker()

In [ ]:
def N_gmratio(fname):
    f=h5py.file(fname,'r')
    m_m0s=np.array(f['m_m0s'])
    N_fc=f['n_fc'].value
    
    Nbins=50
    lowerbound=min(m_m0s)
    upperbound=max(m_m0s)
    #print lowerbound
    #print upperbound
    bins=np.logspace(np.log10(lowerbound),np.log10(upperbound),Nbins)
    midbins=(bins[:-1]+bins[1:])/2
    
    fig=plt.figure(figsize=(8,6))
    ax=fig.add_subplot(1,1,1)
    N_gmratio=ax.hist(m_m0s,
              weights=np.repeat(1./N_fc,
                                len(m_m0s)),
              bins=bins,cumulative=-1)[0]
    ax.set_yscale('log')
    #ax.set_xscale('log')
    plt.clf()
    
    return N_gmratio,midbins

In [ ]:
timestmp=20181026
fname='$home/dat/true_m_12_{}_subhalo.h5'.format(timestmp)
N_gmratio(fname)